# Transfer Learning: MobileNetV2

We will use a pre-trained MobileNetV2 with "imageNet" weights. 

### imports

In [1]:
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt


### Generators for loading and augmenting images

In [2]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                      shear_range=0.1,
#                                      zoom_range=0.1,
#                                      rotation_range=5,
#                                      width_shift_range=0.1,
#                                      height_shift_range=0.1,
#                                      fill_mode='nearest')
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Model

In [6]:
class XrayModel(tf.keras.Model):
    def __init__(self):
        super(XrayModel, self).__init__()

        self.base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),
                                                            include_top=False,
                                                            weights='imagenet')
        for layer in self.base_model.layers:
            layer.trainable = False
        for layer in self.base_model.layers[-10:]:
            layer.trainable = True

        self.pool = MaxPool2D()
        self.flatten = Flatten()
        self.classifier = Dense(3, activation='softmax')
    
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.pool(x)
        x = self.flatten(x)
        outputs = self.classifier(x)
        return outputs
    
model = XrayModel()


## Load the saved model

In [ ]:
model = tf.keras.models.load_model('../models/mobileNetV2')

### Compile and train

In [8]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics='accuracy')


In [9]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.90:
            self.model.stop_training = True


In [10]:
model.fit(train_generator,
         steps_per_epoch=100,
         epochs=5,
         validation_data=test_generator,
         validation_steps=3,
         callbacks=[MyCallback(), 
                    EarlyStopping(monitor='val_loss', mode='min',
                                  verbose=1, patience=5)])

Epoch 1/5
100/100 [==============================] - 77s 751ms/step - loss: 0.5790 - accuracy: 0.9016 - val_loss: 20.6309 - val_accuracy: 0.5729


## Evaluate

In [11]:
model.evaluate(test_generator)

41/41 [==============================] - 26s 641ms/step - loss: 15.8181 - accuracy: 0.6638


[15.818097114562988, 0.6638198494911194]

## Save

In [12]:
model.save('../models/mobileNetV2', save_format='tf')

2022-03-29 10:58:32.696525: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../models/mobileNetV2/assets


/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
